In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Data

## Sequence

**[train/validation/test]_sequences.csv** - the target sequences of the RNA molecules.

- `target_id` - (string) An arbitrary identifier. In train_sequences.csv, this is formatted as pdb_id_chain_id, where pdb_id is the id of the entry in the Protein Data Bank and chain_id is the chain id of the monomer in the pdb file.
- `sequence` - (string) The RNA sequence. For test_sequences.csv, this is guaranteed to be a string of A, C, G, and U. For some train_sequences.csv, other characters may appear.
- `temporal_cutoff` - (string) The date in yyyy-mm-dd format that the sequence was published. See Additional Notes.
- `description` - (string) Details of the origins of the sequence. For a few targets, additional information on small molecule ligands bound to the RNA is included. You don't need to make predictions for these ligand coordinates.
- `all_sequences` - (string) FASTA-formatted sequences of all molecular chains present in the experimentally solved structure. In a few cases this may include multiple copies of the target RNA (look for the word "Chains" in the header) and/or partners like other RNAs or proteins or DNA. You don't need to make predictions for all these molecules; if you do, just submit predictions for sequence. Some entries are blank.


In [8]:
train_sequence = pd.read_csv('./data/train_sequences.csv')
train_sequence.head(5)

,target_id,sequence,temporal_cutoff,description,all_sequences
0,1SCL_A,GGGUGCUCAGUACGAGAGGAACCGCACCC,1995-01-26,"THE SARCIN-RICIN LOOP, A MODULAR RNA",>1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus n...
1,1RNK_A,GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU,1995-02-27,THE STRUCTURE OF AN RNA PSEUDOKNOT THAT CAUSES...,>1RNK_1|Chain A|RNA PSEUDOKNOT|null\nGGCGCAGUG...
2,1RHT_A,GGGACUGACGAUCACGCAGUCUAU,1995-06-03,24-MER RNA HAIRPIN COAT PROTEIN BINDING SITE F...,>1RHT_1|Chain A|RNA (5'-R(P*GP*GP*GP*AP*CP*UP*...
3,1HLX_A,GGGAUAACUUCGGUUGUCCC,1995-09-15,P1 HELIX NUCLEIC ACIDS (DNA/RNA) RIBONUCLEIC ACID,>1HLX_1|Chain A|RNA (5'-R(*GP*GP*GP*AP*UP*AP*A...
4,1HMH_E,GGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU,1995-12-07,THREE-DIMENSIONAL STRUCTURE OF A HAMMERHEAD RI...,">1HMH_1|Chains A, C, E|HAMMERHEAD RIBOZYME-RNA..."


In [7]:
train_sequence_v2 = pd.read_csv('./data/train_sequences.v2.csv')
train_sequence_v2.head(5)

,target_id,sequence,temporal_cutoff,description,all_sequences
0,7TAX_M,CUAAGAAAUUCACGGCGGGCUUGAUGUCCGCGUCUACCUGAUUCAC...,2022-09-21,Cryo-EM structure of the Csy-AcrIF24-promoter ...,>7TAX_1|Chain A|CRISPR-associated protein Csy1...
1,4WF1_CA,AAUUGAAGAGUUUGAUCAUGGCUCAGAUUGAACGCUGGCGGCAGGC...,2014-11-05,Crystal structure of the E. coli ribosome boun...,">4WF1_1|Chains A[auth AA], BB[auth CA]|16S rRN..."
2,8PVA_b,UGCCUGGCGGCCGUAGCGCGGUGGUCCCACCUGACCCCAUGCCGAA...,2023-11-29,Structure of bacterial ribosome determined by ...,>8PVA_1|Chain A|16S rRNA|Escherichia coli (562...
3,8OVE_BB,CAACUGCAGACCGUACUCAUCACCGCAUCAGGUCCCCAAGCAUCGA...,2023-11-29,CRYO-EM STRUCTURE OF TRYPANOSOMA BRUCEI PROCYC...,>8OVE_1|Chain A[auth AA]|SSU rRNA|Trypanosoma ...
4,8JDL_w,UACCUGGUUGAUCCUGCCAGUAGCAUUGCUUGCCAAAGAUUAAGCC...,2023-12-06,Structure of the Human cytoplasmic Ribosome wi...,>8JDL_1|Chain A|mRNA|Homo sapiens (9606)\nUUAU...


In [15]:
test_sequence = pd.read_csv('./data/test_sequences.csv')
train_sequence.head(5)

,target_id,sequence,temporal_cutoff,description,all_sequences
0,1SCL_A,GGGUGCUCAGUACGAGAGGAACCGCACCC,1995-01-26,"THE SARCIN-RICIN LOOP, A MODULAR RNA",>1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus n...
1,1RNK_A,GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU,1995-02-27,THE STRUCTURE OF AN RNA PSEUDOKNOT THAT CAUSES...,>1RNK_1|Chain A|RNA PSEUDOKNOT|null\nGGCGCAGUG...
2,1RHT_A,GGGACUGACGAUCACGCAGUCUAU,1995-06-03,24-MER RNA HAIRPIN COAT PROTEIN BINDING SITE F...,>1RHT_1|Chain A|RNA (5'-R(P*GP*GP*GP*AP*CP*UP*...
3,1HLX_A,GGGAUAACUUCGGUUGUCCC,1995-09-15,P1 HELIX NUCLEIC ACIDS (DNA/RNA) RIBONUCLEIC ACID,>1HLX_1|Chain A|RNA (5'-R(*GP*GP*GP*AP*UP*AP*A...
4,1HMH_E,GGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU,1995-12-07,THREE-DIMENSIONAL STRUCTURE OF A HAMMERHEAD RI...,">1HMH_1|Chains A, C, E|HAMMERHEAD RIBOZYME-RNA..."


## Labels

- `ID` - (string) that identifies the target_id and residue number, separated by _. Note: residue numbers use one-based indexing.
- `resname` - (character) The RNA nucleotide ( A, C, G, or U) for the residue.
- `resid` - (integer) residue number.
- `x_1,y_1,z_1,x_2,y_2,z_2,…` - (float) Coordinates (in Angstroms) of the C1' atom for each experimental RNA structure. There is typically one structure for the RNA sequence, and 
  `train_labels.csv` curates one structure for each training sequence. However, in some targets the experimental method has captured more than one conformation, and each will be used as a potential reference for scoring your predictions. `validation_labels.csv` has examples of targets with multiple reference structures (`x_2,y_2,z_2,` etc.).


In [13]:
train_labels = pd.read_csv('./data/train_labels.csv')
train_labels.head(5)

,ID,resname,resid,x_1,y_1,z_1
0,1SCL_A_1,G,1,13.760,-25.974001,0.102
1,1SCL_A_2,G,2,9.310,-29.638000,2.669
2,1SCL_A_3,G,3,5.529,-27.813000,5.878
3,1SCL_A_4,U,4,2.678,-24.900999,9.793
4,1SCL_A_5,G,5,1.827,-20.136000,11.793


In [12]:
train_labels_v2 = pd.read_csv('./data/train_labels.v2.csv')
train_labels_v2.head(5)

,ID,resname,resid,x_1,y_1,z_1
0,7TAX_M_1,C,1,187.126007,148.246002,210.417999
1,7TAX_M_2,U,2,185.255997,152.968002,204.617996
2,7TAX_M_3,A,3,189.360992,161.802002,205.214996
3,7TAX_M_4,A,4,186.000000,156.595993,209.951996
4,7TAX_M_5,G,5,181.947998,158.186996,213.610992


## Sample Submission

In [16]:
sample = pd.read_csv('./data/sample_submission.csv')
sample.head(5)

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,R1107_1,G,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,R1107_2,G,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,R1107_3,G,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,R1107_4,G,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,R1107_5,G,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Evaulation Metric - TM Score

In [14]:
import os
import re

import pandas as pd
import pandas.api.types


def parse_tmscore_output(output):
    # Extract TM-score based on length of reference structure (second)
    tm_score_match = re.findall(r'TM-score=\s+([\d.]+)', output)[1]
    if not tm_score_match:
        raise ValueError('No TM score found')
    return float(tm_score_match)


def write_target_line(
    atom_name, atom_serial, residue_name, chain_id, residue_num, x_coord, y_coord, z_coord, occupancy=1.0, b_factor=0.0, atom_type='P'
) -> str:
    """
    Writes a single line of PDB format based on provided atom information.

    Args:
        atom_name (str): Name of the atom (e.g., "N", "CA").
        atom_serial (int): Atom serial number.
        residue_name (str): Residue name (e.g., "ALA").
        chain_id (str): Chain identifier.
        residue_num (int): Residue number.
        x_coord (float): X coordinate.
        y_coord (float): Y coordinate.
        z_coord (float): Z coordinate.
        occupancy (float, optional): Occupancy value (default: 1.0).
        b_factor (float, optional): B-factor value (default: 0.0).

    Returns:
        str: A single line of PDB string.
    """
    return f'ATOM  {atom_serial:>5d}  {atom_name:<5s} {residue_name:<3s} {residue_num:>3d}    {x_coord:>8.3f}{y_coord:>8.3f}{z_coord:>8.3f}{occupancy:>6.2f}{b_factor:>6.2f}           {atom_type}\n'


def write2pdb(df: pd.DataFrame, xyz_id: str, target_path: str) -> int:
    resolved_cnt = 0
    with open(target_path, 'w') as target_file:
        for _, row in df.iterrows():
            x_coord = row[f'x_{xyz_id}']
            y_coord = row[f'y_{xyz_id}']
            z_coord = row[f'z_{xyz_id}']

            if x_coord > -1e17 and y_coord > -1e17 and z_coord > -1e17:
                # if True:
                resolved_cnt += 1
                target_line = write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x_coord,
                    y_coord=y_coord,
                    z_coord=z_coord,
                    atom_type='C',
                )
                target_file.write(target_line)
    return resolved_cnt


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Computes the TM-score between predicted and native RNA structures using USalign.

    This function evaluates the structural similarity of RNA predictions to native structures
    by computing the TM-score. It uses USalign, a structural alignment tool, to compare
    the predicted structures with the native structures.

    Workflow:
    1. Copies the USalign binary to the working directory and grants execution permissions.
    2. Extracts the `target_id` from the `ID` column of both the solution and submission DataFrames.
    3. Iterates over each unique `target_id`, grouping the native and predicted structures.
    4. Writes PDB files for native and predicted structures.
    5. Runs USalign on each predicted-native pair and extracts the TM-score.
    6. Computes the highest TM-score per target and returns aggregated results.

    Args:
        solution (pd.DataFrame): A DataFrame containing the native RNA structures.
        submission (pd.DataFrame): A DataFrame containing the predicted RNA structures.
        row_id_column_name (str): The name of the column containing unique row identifiers.

    Returns:
        float: the average highest TM-scores.
    """

    os.system('cp //kaggle/input/usalign/USalign /kaggle/working/')
    os.system('sudo chmod u+x /kaggle/working//USalign')

    # Extract target_id from ID (target_resid)
    solution['target_id'] = solution['ID'].apply(lambda x: x.split('_')[0])
    submission['target_id'] = submission['ID'].apply(lambda x: x.split('_')[0])

    results = []
    # Iterate through each target_id and generate PDB files for both clean and corrupted data
    for target_id, group_native in solution.groupby('target_id'):
        group_predicted = submission[submission['target_id'] == target_id]
        native_pdb = 'native.pdb'
        predicted_pdb = 'predicted.pdb'

        target_id_scores = []
        for pred_cnt in range(1, 6):
            prediction_scores = []
            for native_cnt in range(1, 41):
                # Write solution PDB
                resolved_cnt = write2pdb(group_native, native_cnt, native_pdb)

                # Write predicted PDB
                _ = write2pdb(group_predicted, pred_cnt, predicted_pdb)

                if resolved_cnt > 0:
                    command = f'/kaggle/working/USalign {predicted_pdb} {native_pdb} -atom " C1\'"'
                    usalign_output = os.popen(command).read()
                    prediction_scores.append(parse_tmscore_output(usalign_output))

            target_id_scores.append(max(prediction_scores))
        results.append(max(target_id_scores))

    return float(sum(results) / len(results))